# Megaleiloes PDF/Image Scraper
#### Made By- Uday S.
#### For- Raphael G.

## I. Importing Libraries

In [1]:
#importing drivers
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
#importing beautiful soup
from bs4 import BeautifulSoup
# Import requests
import requests
  
#importing the necessary libraries
import pandas as pd, numpy as np
#importing json package to read the data
import json

#importing os module
import os
from os.path import basename
#importing time
import time
#importing pathlib
import pathlib
#importing string
import string

## II. Defining Variable

In [2]:
#defining a function to find the position of a substring's nth instance in a string
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

## III. Scraping Page Links

In [3]:
# Using Selenium to define driver
driver = webdriver.Chrome(ChromeDriverManager().install())
url= "https://www.megaleiloes.com.br/imoveis"
driver.get(url)
p_db=""
for i in range(13):
    time.sleep(5)
    p_element = driver.find_element_by_xpath('//*[@id="w0"]/div[1]')
    p_db= p_db + " " +p_element.get_attribute('innerHTML')# Getting attributes of an element
    try:
        click_element = driver.find_element_by_xpath('//*[@id="w0"]/div[2]/ul/li[11]/a')
        click_element.click()
    except:
        pass
url_df= pd.DataFrame()
url_df["url"]= [None] * 584

for i in range(584):
    try:
        url_df["url"][i]= p_db[find_nth(p_db,'class="card-title" href="',i+1)+25:find_nth(p_db,'" data-pjax="0">',(3*i)+1)]
        a=i
    except:
        print(i)

## IV. Defining Columns And Directories

In [4]:
#Defining the Name column
url_df["Name"]=""
#Using for loop to iterate through the rows to fill the Name column with the appropriate name to use as file names
for i in range(len(url_df)):
    url_df["Name"][i]= url_df["url"][i][find_nth(url_df["url"][i],"/",7)+1:]

In [5]:
#Defining Parent Directory
parent_dir = os.getcwd()
# Directory
directory = "Megaleiloes"
# Directory paths
path = os.path.join(parent_dir, directory)
# Create the directory
if not os.path.exists(path):
    os.mkdir(path)
else:
    pass

## V. Downloading PDFs

In [6]:
for n in range(len(url_df)):
    # URL from which pdfs to be downloaded
    url = url_df["url"][n]

    # Requests URL and get response object
    response = requests.get(url)

    # Parse text obtained
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all hyperlinks present on webpage
    links = soup.find_all('a')

    i = 0
    
    
    d1= url_df["Name"][n]
    # Directory paths
    path1= os.path.join(path, d1)

    # Create the directory
    if not os.path.exists(path1):
        os.mkdir(path1)
    else:
        pass
    # From all links check for pdf link and
    # if present download file
    for link in links:
        if ('.pdf' in link.get('href', [])):
            i += 1
            print("Downloading file: ", n,": ",i)

            # Get response object for link
            response = requests.get(link.get('href'))
            if i==1:
                j="Edital"
            elif i==2:
                j="Laudo de avaliação"
            elif i==3:
                j="Matrícula"
            # Write content in pdf file
            pdf = open(directory+"/"+d1+"/"+j+".pdf", 'wb')
            pdf.write(response.content)
            pdf.close()
            print("File ", n,": ",i, " downloaded")

## VI. Downloading Images

In [8]:
for n in range(len(url_df)):
    # URL from which jpgs to be downloaded
    url = url_df["url"][n]

    # Requests URL and get response object
    response = requests.get(url)

    # Parse text obtained
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all hyperlinks present on webpage
    links = soup.find_all('img')

    i = 0


    d1= url_df["Name"][n]
    # Directory paths
    path1= os.path.join(path, d1)

    # Create the directory
    if not os.path.exists(path1):
        os.mkdir(path1)
    else:
        pass
    # From all links check for jpg link and
    # if present download file
    for link in links:
        if ".jpg" in str(link):
            i += 1
            
            print("Downloading file: ", n,": ",i)
            # Get response object for link
            response = requests.get(link.get("src"))
            # Write content in jpg file
            jpg = open(directory+"/"+d1+"/"+str(i)+".jpg", 'wb')
            jpg.write(response.content)
            jpg.close()
            print("File ", n,": ",i, " downloaded")